# RSA演算法

[米勒-拉賓質數判定法](https://zh.wikipedia.org/wiki/%E7%B1%B3%E5%8B%92-%E6%8B%89%E5%AE%BE%E6%A3%80%E9%AA%8C)

In [1]:
import base64
import random


class RSA():
    def __init__(self, bits: int, e: int = 65537):
        self.e = e
        self.bits = bits
        self.__rsa(bits, e)
        
    def __rsa(self, bits, e):
        for _ in range(2):
            while True:
                prime_bits = '1' + ''.join([random.choice(['0', '1']) for __ in range(bits//2-2)]) + '1'
                prime = int(prime_bits, 2)
                if self.__miller_rabin(prime, 3):
                    if _ == 0:
                        self.p = prime
                    else:
                        self.q = prime
                    break
                else:
                    continue
        self.n = self.p * self.q
        phi_n = (self.p - 1) * (self.q - 1)
        self.d = self.__euclid(self.e, phi_n)

    def __miller_rabin(self, n, k):
        if n == 2 or n == 3:
            return True

        if n % 2 == 0:
            return False

        r, s = 0, n - 1
        while s % 2 == 0:
            r += 1
            s //= 2

        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, s, n)
            if x == 1 or x == n - 1:
                continue
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        return True
    
    def __euclid(self, e: int, n: int) -> int:
        r1 = n
        r2 = e
        t1 = 0
        t2 = 1
        while r1 != 1:
            q = r1 // r2
            r = r1 % r2
            r1 = r2
            r2 = r
            t = t1 - (q * t2)
            t1 = t2
            t2 = t
        if t1 < 0:
            t1 += n
        return t1
    
    def exponentiation(self, base, exp, N):
        t = 1
        while(exp > 0): 
            if (exp % 2 != 0):
                t = (t * base) % N
            base = (base * base) % N
            exp = exp // 2
        return t % N
        
    def encrypt(self, plain_text: str) -> tuple:
        cipher = []
        for char in plain_text:
            cipher.append(base64.b64encode(self.exponentiation(int.from_bytes(char.encode('big5'), 'little'), self.e, self.n).to_bytes(self.bits//8, byteorder='little')).decode('utf-8'))
        return tuple(cipher)
            
    def decrypt(self, cipher: tuple) -> tuple:
        msg = []
        for char in cipher:
            msg.append((self.exponentiation(int.from_bytes(base64.b64decode(char.encode('utf-8')), 'little'), self.d, self.n)).to_bytes(2, 'little').decode('big5'))
        return tuple(msg)

In [2]:
rsa_key = RSA(2048)

In [3]:
print(f'\n私鑰：\033[33m{rsa_key.d}\033[0m\n\n公鑰：\033[33m{rsa_key.e}\033[0m')


私鑰：9828801162294202516249089844878591782589184364158640517542566889301093726799529224082721754748107867260275181450965357973239358338472248485562111314052714521203309606778262911281065976441357663961600538952179401283081694215525460489342112978660672085256399495411687105207075987539344372661597111086109748223703436460623902278068028314799814620868297204261707971102949989882563255991544639053107782938463159027775200914329697202860145267491590178297555511240405040771059519090937769712076742383624558182153271608999319046415207220496414019797455530597830753330492260160163828127219234307889971122677174581999028650913

公鑰：65537


In [4]:
plain_text = "All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

In [5]:
cipher = rsa_key.encrypt(plain_text)
print(''.join(cipher))

EEqwwSGMykbHEV0UgORFoVSeaoiAa4GXZDagk35kju8I5HxziQ87720iIsU/jKgq0D46QJEKyRFyvDArjHNW/QEgVMdyLlgboOCI5E9xzwlGZOSe/CTStlGfAz6vB4DsuOlszFcrHsnvVqHdZQpi9oDye69EIz4kKHoRtZ4R7d06FEvlO7AMdR8IdEhB4A1qyynKytXEnprOudic7/kWGCj/Qh8aF7LTOn/sIqYLyeIUQ9vZxBl/e//EsfN6/KTDekrMk4GbtNU2ZT4nkp3lCZJVkS6wRJp09ToxKUp+TpU/m1j5JCq9Z+7KB/iG9+UG00fNfx6EBsyinpfS+fwmDQ==x0XnoiUyFeKryXv3Htnn6tN9GeuPmxma4KXXch044LikXjoAyZ0Z5h9S83/bGmnhIqXBscstey6PZPhXihCKoLAaWyCs57mzjDqRTTYxiYqo0W8IRXBt0ClGhxwS8ILkYiR2+jUJ1lD1sZr35Qw3Gpl/GMN58NLKmvo1WfXZ3wD0jrvpzpzIl80v0CAUPowe58lQQWBkzLLAZZJP/xXHc9VqAFRS1aI1S5H7NWDR6E4o1XKIf/pLeOiYQD9HnZfrhq9FapRu4Ru5fQ0AZpzkIWyPjV5+RSUBGN85RKB78AE7mVECcaJvsibHSqZ/ndEp/UKuhRC270tlcSBb9+SdDQ==x0XnoiUyFeKryXv3Htnn6tN9GeuPmxma4KXXch044LikXjoAyZ0Z5h9S83/bGmnhIqXBscstey6PZPhXihCKoLAaWyCs57mzjDqRTTYxiYqo0W8IRXBt0ClGhxwS8ILkYiR2+jUJ1lD1sZr35Qw3Gpl/GMN58NLKmvo1WfXZ3wD0jrvpzpzIl80v0CAUPowe58lQQWBkzLLAZZJP/xXHc9VqAFRS1aI1S5H7NWDR6E4o1XKIf/pLeOiYQD9HnZfrhq9FapRu4Ru5fQ0AZpzkIWyPjV5+RSUBGN85RKB78AE7mVECcaJvsibH

In [6]:
''.join(rsa_key.decrypt(cipher)).replace('\x00', '')

"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

---

# 數位信封

## 產生對稱式金鑰

### 沒有 IV 會導致前 16 個字無法解碼

In [7]:
def generate_aes_key(bytes: int) -> str:
    key = b''.join([(random.randint(0, 0xFF).to_bytes(1, 'little')) for _ in range(bytes)])
    return key

aes_key = generate_aes_key(32)
IV = generate_aes_key(16)
print(aes_key, IV, sep='\n')

b'[\xd0p\xe4d\x10\xc6zk\x86\x91+^\xab/\x96\x02\x97\xb9\xfbx\xa9\xc6b;p)\xcd\x13b\xb1H'
b'!\xcb\xc10V\x9f\xcc\x06)\xf7RP\xd0\x13\xd7\x19'


## 加密明文

In [8]:
plain_text = "All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

### Padding

In [9]:
def padding(text):
    if type(text) is str:
        text = text.encode('utf8')
    elif type(text) is bytes:
        pass
    else:
        raise TypeError('請輸入字串或位元組資料')
    if len(text) % 16 != 0:
        text += ('\0'.encode('utf8') * (16 - (len(text) % 16)))
        
    return text

In [10]:
plain_text = padding(plain_text)
plain_text

b"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use.\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [11]:
len(plain_text)

128

In [12]:
from Crypto.Cipher import AES

cipher_aes = AES.new(aes_key, AES.MODE_CBC, IV)
try:
    cipher = cipher_aes.encrypt(plain_text)
except TypeError:
    cipher = cipher_aes.encrypt(plain_text.encode())
except Exception as e:
    print(e)

In [13]:
cipher

b'^\xb6\xf3\xb4\xff%\xbbG\r7\x9a:\xecJ\xb7y9\x90*\xc1\x9a\x91\xd0\xe5\x8f\n*L\xeb\x08\xdeQ\x00\x8c\x0e\t\xe6wU\xb3\xd8\xfcB\xb0\x9b\xf4\x1b\xd0xd$\x92\x1f\xb7@\xc9\xdd\x0b\xa8r&N\xa4\x18u\x9ai\x8a\x0cF#\xd7\xa1q\xef\xa08m\xfa;m\x06\xad\x06\xae\xc8\x9c\x15\xc83\xb7\x07x\xf8Zb\xa7\xef\x8f\xaf\xda\xec)\xf3Y\xec\x9d\x10\xc2\xbb!1\xfc,\x98$\x18\x0b#\x8cY\xf27\xd2ey\xe1Y'

## 數位信封RSA演算法

In [14]:
class DigitalRSA(RSA):  
    def encrypt(self, secret_key: bytes):
        cipher = []
        for B in secret_key:
            cipher.append(base64.b64encode(self.exponentiation(B, self.e, self.n).to_bytes(self.bits//8, byteorder='little')))
        return cipher
    
    def decrypt(self, encrypted_secret_key: list):
        decrypted_key = []
        for B in encrypted_secret_key:
            decrypted_key.append(self.exponentiation(int.from_bytes(base64.b64decode(B), 'little'), self.d, self.n).to_bytes(1, 'little'))
        return decrypted_key

## 加密AES金鑰

In [15]:
d_rsa = DigitalRSA(2048)
key = d_rsa.encrypt(aes_key)

In [16]:
print(f'\n私鑰：\033[33m{d_rsa.d}\033[0m\n\n公鑰：\033[33m{d_rsa.e}\033[0m')


私鑰：17778136736898578935514190309487876752878320910796045712383650483650567834678847987074286706203937058976502905168065713764744079403429236622635917614795012023812506979940468978899305120397055892012133852742151949402546791952427719466469759349887786209340905695908160725567060580103471993212871197745998126251243478571625596220633574292213019628077793075753111627236641605731521805630513726133658827172499282078048060836445813881478935999233929564257716308707169009991879425289563484391622349800201672007658298694742018122131323606975867612437926799620521900104695901076570014198457359548289559664166989769395495951761

公鑰：65537


In [17]:
b''.join(key)

b'cv3BUs5ydguOeKfRR9ZpvXF5prj1BFrRyz37iYL/tn/790jGZdpQUfSGJYiN0156vgm4/9o25C06V04PjWpscpHAUDDNysXxuFCa5C2pfwFuzwDwUzbFQbf6jaLZIDrUTPgkw9OD7Uw9XP1yszMcqkHWSR8e5UpSbUuyPFyTHtxSlVunSfBV1/R6H3gf7Rw7GVbNBSCLBx7CNkJ6nOt1x3z8H437VBTcW2BC2KffjOGxUWUGWCJSqS9Xm4nBSVcHs8nfJP/bnDZPaXS8PBzLRKNwmNg67P7oI9wsWzgAUtzY/T2TxMPy91DoEGgsF21mmlykzA1Je8JCskjc2ipvFg==0ncIJ9wPlLb9zTNhBk0sowKepSuYReum/4PK/YJliGThnukjrisw5710dKpTkN3uwLdpfy0hMEo0ia8ClRG0vZL6HrAL5p7+ofmi0T/9kALn6x0AVVGcv+S+UwGWzvgcjbC29kQPKcoGRr8DBIVhcf3n+FGM4F6SCEO29gM3Qk61ZZbBQeCfR2pXBkU+Y5EWm9aukpmqdcz8XHAscDZ7zogahJcW8dclcoOQILpnnvfRHocb2cN7z+hg64exFNUbUtrCTOFiLElou9VnVCr/A2Jnd4WnKedAic1ztRJFubs7S4NezndY0zZgrb0N/IFIykkiGVK63vp4wvc9Y+mFpA==9OQr7oOxR/33FfPUJGTt/6JlWMrIqStPCyhw/jST9XyuAXdO0p0/SAgpR2wZnrl2pYhYp6VUJh0MwgY73lYKXxwrCI2ETuwGeyjRYvjq2xmfsUuB4hilaxX/+LdUzM8MaKk4nPsKjvlp5HedgWcUC1qeHT8Irg0y21vTW7leiQM1B3XQR5D6OnSkvLYN0aCHDgEgx11MKi1EZUiMio+IrbspiGBRDwcBmwdzbF2aZ1mAc0ieQLA3g00shx7+nqe11EKhgZPSMnZHeJd1u1sepoQdyjxlkztYqSi3dayj4C3atxrE3IdyE3

## 解密AES金鑰

In [18]:
decrypted_aes_key = bytes()
for _ in d_rsa.decrypt(key):
    decrypted_aes_key += _
print(f'{"Decrypted secret key:":>12} \033[33m{decrypted_aes_key}\033[0m\n{"Original secret key:":>{len("Decrypted secret key:")}} \033[33m{aes_key}\033[0m\n\nEqual? {decrypted_aes_key == aes_key}')

Decrypted secret key: b'[\xd0p\xe4d\x10\xc6zk\x86\x91+^\xab/\x96\x02\x97\xb9\xfbx\xa9\xc6b;p)\xcd\x13b\xb1H'
 Original secret key: b'[\xd0p\xe4d\x10\xc6zk\x86\x91+^\xab/\x96\x02\x97\xb9\xfbx\xa9\xc6b;p)\xcd\x13b\xb1H'

Equal? True


## 使用解密之AES金鑰解密秘文

In [19]:
decryptor = AES.new(decrypted_aes_key, AES.MODE_CBC, IV)

decryptor.decrypt(cipher).decode('utf8').replace('\x00', '')

"All I ask is if this is my last night with you, hold me like I'm more than just a friend. Give me a memory I can use."

---

# 長文章數位信封

In [20]:
import requests as rq
from bs4 import BeautifulSoup as bs


res = rq.get('https://mrmad.com.tw/apple-redesign-macbook-air-2021-new-color')
soup = bs(res.text, 'lxml')

paragraph = []
for p in soup.find_all('p')[: -3]:
    p = p.text
    if p != '':
        paragraph.append(p)
        
paragraph = '\n'.join(paragraph)
print(paragraph)

蘋果爆料達人 Jon Prosser 在最近獨家洩密新一代M2款 MacBook Air 將採用「彩色」外觀設計，與蘋果春季發表會推出的 24吋 iMac 顏色相同，規格與外觀也會迎來4大方面改進。
 
Jon Prosser 過去就曾準確爆料 iMac 2021年款會加入多種新顏色，這次又從可靠來源得知，下一代 M2 款 MacBook Air 外觀預計也會加入多款顏色，顏色與 M1 款 iMac 完全相同，共會有7種配色，分別是藍色、 綠色、 粉紅色、 銀色、 黃色、 橙色、 紫色。
且 Jon Prosser 也從蘋果內部知情人士得知，當前 Apple 內部已經有藍色 MacBook Air 原型機。
至於 2021 MacBook Air 新配色會如同底下這幾張模擬圖，主要是採用淡色系，另還會加入 MagSafe 磁吸充電。
 
至於 Apple 打算替 iMac 和 MacBook Air 加入多款彩色選擇，最主要是替產品等級區隔，對於一般消費者而言，追求的並非是效能，而是外觀和便利性，推出多顏色更能提升用戶接受度，對於需要高效能專業級別的用戶，硬體規格需求明顯大於顏色，也是會有黑色或灰色 Pro 級別款式可供選擇。
從去年 10月蘋果發表會，蘋果推出首款多顏色 iPad Air 後，明確顯示要讓產品走向繽紛色，同時要以顏色來區隔等級。
如同 M1款 iMac 和 iPhone 普通與專業款差異，對於一般型消費市場，則是以多顏色來搶攻普通用戶目光和荷包，同時也有專業 Pro 級別款式，則是以原色或深色款來突顯產品的強勁性能與規格。
過去彭博社（Bloomberg）記者 Mark Gurman 也曾爆料，蘋果已經準備研發新一代更輕薄的 MacBook Air ，預計會在 2021 下半年推出，連同蘋果分析師郭明錤也曾指出，會有一款低階 MacBook 會搭載 mini-LED 螢幕。
如從目前多方消息來看，新款 MacBook Air 2021 預計採用多顏色、M2處理器、MagSafe 磁吸充電和厚度將變更薄為主要特色，至於 mini-LED 螢幕預計會是給 MacBook Pro 系列才會搭載。
更新：MacBook Air 2021機身曝光，搶先看6大外觀改進和發表時間


In [21]:
from Crypto.Cipher import AES

class DigitalEnvelope():
    def __init__(self, key_bytes: int = 32, IV: bool = True):
        self.aes_key = self.__generate_aes_key(key_bytes)
        self.IV = self.__generate_aes_key(16) if IV else None
        print(f'{"AES secret key:":>12} \033[33m{self.aes_key}\033[0m\n{"IV:":>{len("AES secret key:")}} \033[33m{self.IV}\033[0m')
    
    def __generate_aes_key(self, key_bytes: int) -> bytes:
        key = b''.join([(random.randint(0, 0xFF).to_bytes(1, 'little')) for _ in range(key_bytes)])
        return key
    
    def __padding(self, text):
        if type(text) is str:
            text = text.encode('utf8')
        elif type(text) is bytes:
            pass
        else:
            raise TypeError('請輸入字串或位元組資料')
        if len(text) % 16 != 0:
            text += ('\0'.encode('utf8') * (16 - (len(text) % 16)))

        return text
    
    def encrypt(self, plain_text, secret_key, IV, mode = AES.MODE_CBC):
        plain_text = self.__padding(plain_text)
        cipher_aes = AES.new(secret_key, mode, IV)
        try:
            cipher = cipher_aes.encrypt(plain_text)
        except TypeError:
            cipher = cipher_aes.encrypt(plain_text.encode('utf8'))
        except AttributeError:
            cipher = cipher_aes.encrypt(plain_text)
        return cipher.decode('ISO-8859-1')

    def decrypt(self, cipher_text, secret_key, IV, mode = AES.MODE_CBC):
        decryptor = AES.new(secret_key, mode, IV)
        return decryptor.decrypt(cipher_text.encode('ISO-8859-1'))
            
            
class CipherKey(RSA):  
    def encrypt(self, secret_key: bytes) -> tuple:
        cipher_key = []
        for B in secret_key:
            cipher_key.append(base64.b64encode(self.exponentiation(B, self.e, self.n).to_bytes(self.bits//8, byteorder='little')))
        return tuple(cipher_key)
    
    def decrypt(self, encrypted_secret_key: tuple):
        decrypted_key = []
        for B in encrypted_secret_key:
            decrypted_key.append(self.exponentiation(int.from_bytes(base64.b64decode(B), 'little'), self.d, self.n).to_bytes(1, 'little'))
        s = bytes()
        for _ in decrypted_key:
            s += _
        return s

In [22]:
plain_text = paragraph
DE = DigitalEnvelope()
aes_key = DE.aes_key
IV = DE.IV

AES secret key: b'\xfb%L\xea\xaa\x82\xd0\xb6\xbc\x8b\x16X\xeb\xa8%\xf6\xa1\x00\xe4Lv\x93\x19\xf7o\xf7R-\x06\xb1\xcaP'
            IV: b'\x9bb\x02\x8c\xed\xdc&/#+\xa1\x94GsN\xe6'


In [23]:
cipher_text = DE.encrypt(plain_text, aes_key, IV)
cipher_text

'\x98\x08<[Uoß\x06T\x17\x8a@+ý¡ìÙ\x1c\x85lsnyÛ\x7f\x18¡ú\x8c1 Ç¼ÿ;\x96Ê×ñ0\xa0ÊË´ö¶wMTP³Íf"[\x93\x10\x9a³¨Ù\x19\x91¤d¢\x81OwÐ>K\x17\x9awÞd1·Jn\x02´×!\x00ÉÞ)RÜå\n\x14¯!È7°®A\x85Å&!,Ð*àé\x86\x926²,\x99ÈÚIS\x9dc§õ¥\x1fW¬VùÕçg«`òMzëb\x0eayúm\x17F÷\x98£\\ó·íÁnÝåÎþ\x8egÀ\x8b¦\x1e\x12Y·yõÐe\xa0ù©çúº"1ä\\\x88\tÛ¦»\x02õ\x8bö\xa0É\x82\x06R\x91\x9fKÏþ\x81Àa5\x1bHÌïD¸[E\x9e\x8fÁV¬tÜå\x85ß¤Ý\x9fþvy\x96°iE\x9d¶ä¥ \x9dyUÌa}¿H8\x82{:Æ\x8c-wÍ\x94ÓKv\x16bG\x14i\x08X|\x9aÜ\x7fÔ}\x1aë.NÁØ}\xad\x9aûþ(|l\x0cÈÑ\x93U\\w¿g&Ú\x17Ñ¢Ün8\tQ*\x00`\x9d²\x04\x9a\n\x8b¨\x15QPB\'Æ®¤\x8aÒ\x8b¯\x0fÝ~¥%qúi\x08l¸¦½ÝÜ\x10C\x1b~r´r\x9aH©\x02Ì\x1c³!5\x96\x82\x06ìf0ó;\n\x91\x8bà$\x8bÍ¦Ú9\x8eo¾\x84XhUÐ\x07ÌXiJ\x7fMþ¹\x16Ùè\x19Û-ÊKaéÃ¦\x80©À\x07\x1dÄaI\x02\x9b+ÚB\x9a¯Ý\x83G¬§IJºô\x8fM\x93Râì\x84°\x91\x82®ä\x10Êæ²¶¡\x8b\x8a\x9f\x04àSÔ\x06¢b\x9fòp\x19©Õ\x82\x9aý~C¦\x10\x83x\x01ñàfÆÎ}oôS*¸Tå¿\x93\x88Àæøk¡j\x85É\x18d´Ó\x0fÈu\x13\x8e#Ã`8}¶\x98¶N_ÏhP\xadR¼\x98Ô¸ÑL¼¦Ì\x80\x1f9\x89Î\x077ÂÉÁµíeÖ\x88_\x83\x12gç`É\x92\x95äè\x05ë/ûÓÅÑi¶\x96

In [24]:
d_rsa = CipherKey(2048)
encrypted_aes_key = d_rsa.encrypt(aes_key)
b''.join(encrypted_aes_key)

b'XcFplxpsEdORjr9FuFVFliPJgI5Xb/74aHmxJyY++mte4T66pqP6onbam/D6WlqMnTKccDzFjblCD0OEfJvVFouMefc9FMSdkzAg+MZ3oisZ31Ugc2wcqMtKHrbWYq1cRkhot9GsQR0Wvi0vJzQkT/oFzbN2zg10d8RghrHTkEBTw/qVLVLWtEOPwvt5rIBva86acVUV7n8GPCgaZup+3gexNIF+y3QfljcDFZeqtWv0+xgc+Ohs+dSTqNUvxBwBmRR7HyzmIlite+187KpzkAD8Pe5Lk/Rb9ybj9gqQGwF/GhNlZbMbrgX/G2mzip4tF6ad/VGDYTRdsOLzR7bBKw==AA5yg85/MR6anuBqzeImRSV3NXXDqU90RMrvYuj/stuCwlqdXfp2/ongPlcMmetHGMl5uZ6LjQBCBH1b+JwSHh+T/FTsDi50y8EQEva0GtOEFs50Uh2KlyG/orXdwiIwYemmW1v0COmhRcqWuM0R4y/MhGXj/iXCFAB/9svLl6Ig+6gJagm6HV/mU7cpVeqSRfQRhB/bMiXJHX+PjUOxpYqt/HSn6/XldM7z99OvceeGtEHpPXw9+enDCAtixu3P05v3sDjmMUOGvhTgRQe7flTd66l1Sb/s4NEisiF42Ec9CUuMmVomzH66nYn8zpMBq6PrTjyryLkK4bbp+i/VXQ==rAIDTN2cbGoPRg/IOx1k68fs9EOt4m9vAEX/2QG6U65fzHO7l0vlFt7bpsaO5090gGbcMU1JmveTjWawLyCbYVFDm1kgIN+pnKpwmDb2rOz/49l7XL/P+A1GiGcvFlsX7+jD7CG+9aa0CFB73lDqYFBZaNdG8Jhc2x+yi+kHasEiAUg/DJ77IkwIVZya7sZ76ucX1kIp/rb/nU7itRltmPraTouWlaibhIt3O1zrOjed/Goq9IXb3MJ8OQdUko4TAtmXc9RKWEZcsmGIW2XUGoxN0EzFnA6bpmnFajQfsm9WNT6FCDkrnh

In [25]:
decrypted_aes_key = d_rsa.decrypt(encrypted_aes_key)
print(f'{"Original AES key":>{len("Decrypted AES key")}}: \033[33m{aes_key}\033[0m\nDecrypted AES key: \033[33m{decrypted_aes_key}\033[0m')

 Original AES key: b'\xfb%L\xea\xaa\x82\xd0\xb6\xbc\x8b\x16X\xeb\xa8%\xf6\xa1\x00\xe4Lv\x93\x19\xf7o\xf7R-\x06\xb1\xcaP'
Decrypted AES key: b'\xfb%L\xea\xaa\x82\xd0\xb6\xbc\x8b\x16X\xeb\xa8%\xf6\xa1\x00\xe4Lv\x93\x19\xf7o\xf7R-\x06\xb1\xcaP'


In [26]:
print(DE.decrypt(cipher_text, decrypted_aes_key, IV).decode('utf8').rstrip('\x00'))

蘋果爆料達人 Jon Prosser 在最近獨家洩密新一代M2款 MacBook Air 將採用「彩色」外觀設計，與蘋果春季發表會推出的 24吋 iMac 顏色相同，規格與外觀也會迎來4大方面改進。
 
Jon Prosser 過去就曾準確爆料 iMac 2021年款會加入多種新顏色，這次又從可靠來源得知，下一代 M2 款 MacBook Air 外觀預計也會加入多款顏色，顏色與 M1 款 iMac 完全相同，共會有7種配色，分別是藍色、 綠色、 粉紅色、 銀色、 黃色、 橙色、 紫色。
且 Jon Prosser 也從蘋果內部知情人士得知，當前 Apple 內部已經有藍色 MacBook Air 原型機。
至於 2021 MacBook Air 新配色會如同底下這幾張模擬圖，主要是採用淡色系，另還會加入 MagSafe 磁吸充電。
 
至於 Apple 打算替 iMac 和 MacBook Air 加入多款彩色選擇，最主要是替產品等級區隔，對於一般消費者而言，追求的並非是效能，而是外觀和便利性，推出多顏色更能提升用戶接受度，對於需要高效能專業級別的用戶，硬體規格需求明顯大於顏色，也是會有黑色或灰色 Pro 級別款式可供選擇。
從去年 10月蘋果發表會，蘋果推出首款多顏色 iPad Air 後，明確顯示要讓產品走向繽紛色，同時要以顏色來區隔等級。
如同 M1款 iMac 和 iPhone 普通與專業款差異，對於一般型消費市場，則是以多顏色來搶攻普通用戶目光和荷包，同時也有專業 Pro 級別款式，則是以原色或深色款來突顯產品的強勁性能與規格。
過去彭博社（Bloomberg）記者 Mark Gurman 也曾爆料，蘋果已經準備研發新一代更輕薄的 MacBook Air ，預計會在 2021 下半年推出，連同蘋果分析師郭明錤也曾指出，會有一款低階 MacBook 會搭載 mini-LED 螢幕。
如從目前多方消息來看，新款 MacBook Air 2021 預計採用多顏色、M2處理器、MagSafe 磁吸充電和厚度將變更薄為主要特色，至於 mini-LED 螢幕預計會是給 MacBook Pro 系列才會搭載。
更新：MacBook Air 2021機身曝光，搶先看6大外觀改進和發表時間
